In [1]:
import random
import numpy as np
import pandas as pd
import manage_data as md
import preprocessing as pp
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn import metrics

In [143]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues, filename='cm'):
    plt.figure(figsize=(8,8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=20, fontstyle='oblique')
    #plt.colorbar()
    ax = plt.subplot()
    tick_marks = np.arange(2)
    labels = ['Sticking', 'Fine']
    plt.xticks(tick_marks, labels, rotation=45, fontsize=20)
    plt.yticks(tick_marks, labels, fontsize=20)
    plt.tight_layout()
    plt.ylabel('True label', fontsize=20)
    plt.xlabel('Predicted label', fontsize=20)
    width, height = cm.shape
    for x in xrange(width):
        xx = np.round(x, 2)
        for y in xrange(height):
            yy = np.round(y, 2)
            valeur = str(int(np.round(100 * cm[x][y],0)))+'%'
            ax.annotate(valeur, xy=(y, x), 
                        horizontalalignment='center',
                        verticalalignment='center',
                        size=30)
    plt.savefig('./images/cm_one/' + filename + '.png')
    #plt.text(0.1, 0.1,'Caption')

def concat_df(coil_list, filename='peaks.h5'):
    big_df = md.importPeak(coil_list[0], filename).dropna()
    for coil in coil_list[1:]:
        df = md.importPeak(coil, filename).dropna()
        big_df = pd.concat([big_df, df], ignore_index=True)
    return big_df

def create_DB(df):
    feature_col = ['Ximf0', 'Ximf1', 'Ximf2', 'Yimf0', 'Yimf1', 'Yimf2', 'thickness', 'speed']
    X = df[feature_col].values
    Y = df[['sticking']]
    Y = np.array(Y.sticking.values[:], dtype=bool)
    return X, Y

def random_sample(coil_list):
    df = concat_df(coil_list)
    sticking_index = list(df[df.sticking==True].index)
    non_sticking_index = list(df[df.sticking==False].index)
    group_of_items = non_sticking_index               # a sequence or set will work here.
    num_to_select = len(df[df.sticking==True])        # set the number to select here.
    list_of_random_items = random.sample(group_of_items, num_to_select)
    list_of_random_items
    ind = sticking_index+list_of_random_items
    return df.loc[ind]

def random_coils_selection(coil_list, ratio=0.5):
    aux  = list(coil_list)
    random.shuffle(aux)
    n = int(ratio * len(aux))+1
    return aux[:n], aux[n:]

def result_per_coil(coil_list, estimator):
    sum_true = []
    sum_predict = []
    for coil in coil_list:
        df_test = concat_df([coil])
        X_test, Y_true = create_DB(df_test)
        Y_predict = estimator.predict(X_test)
        sum_true.append(sum(Y_true))
        sum_predict.append(sum(Y_predict))
    d = {'truth':sum_true, 'prediction':sum_predict}
    result = pd.DataFrame(d, index=coil_list)
    return result, np.array(sum_true), np.array(sum_predict)

def sticking_coils_finder(coil_list):
    sum_true = []
    coils = []
    for coil in coil_list:
        df_test = concat_df([coil])
        X_test, Y_true = create_DB(df_test)
        sum_true.append(sum(Y_true))
    for coil, sumtrue in zip(coil_list, sum_true):
        if sumtrue>0:
            coils.append(coil)
    fine_coils = sorted(list(set(coil_list) - set(coils)))
    return sorted(coils), sorted(fine_coils)

def df_compare(df=df):
    print('     COLLANT            NON COLLANT  ')
    print(' ')
    
    print('Fréquence moyenne du pic')
    for i in range(3):
        print(df.iloc[:,i][df.sticking==1].mean(), df.iloc[:,i][df.sticking==0].mean())
    print(' ')
    
    print('Ecart type')
    for i in range(3):
        print(df.iloc[:,i][df.sticking==1].std(), df.iloc[:,i][df.sticking==0].std())
    print(' ')
    
    print('Amplitude moyenne du pic')
    for i in range(3,6):
        print(df.iloc[:,i][df.sticking==1].mean(), df.iloc[:,i][df.sticking==0].mean())
    print(' ')
    
    print('Ecart type')
    for i in range(3,6):
        print(df.iloc[:,i][df.sticking==1].std(), df.iloc[:,i][df.sticking==0].std())
    print(' ')
    
    print('Vitesse')
    i = 7
    print(df.iloc[:,i][df.sticking==1].mean(), df.iloc[:,i][df.sticking==0].mean())
    print(df.iloc[:,i][df.sticking==1].std(), df.iloc[:,i][df.sticking==0].std())
    print(' ')
    
    print('Epaisseur')
    i = 9
    print(df.iloc[:,i][df.sticking==1].mean(), df.iloc[:,i][df.sticking==0].mean())
    print(df.iloc[:,i][df.sticking==1].std(), df.iloc[:,i][df.sticking==0].std())
    return

# About DB

In [172]:
#peakDB = 'peaks_1s_1_20Hz.h5'
peakDB = 'peaks_1s_1_100Hz.h5'
#peakDb = 'peaks_1s_2_6Hz.h5'
peakDB = 'peaks_0.25t_16_200.h5'
#peakDB = 'peaks.h5'
all_coils = list( set(range(88)) - set([49, 31]) )
df = concat_df(all_coils, peakDB)
k = 1027
df.head(1)

,Ximf0,Ximf1,Ximf2,Yimf0,Yimf1,Yimf2,coil,speed,sticking,thickness,time_begin,time_end
0,25.59287,26.854491,28.116111,23.204153,3.633037,3.752148,12,1022.035352,False,0.77,21.0,21.25


# Statistics

## Sans normalisation

In [169]:
peakDB = 'peaks_1s_1_100Hz.h5'
df = concat_df(all_coils, peakDB)
df_compare()

     COLLANT            NON COLLANT  
 
Fréquence moyenne du pic
(18.833744588582061, 16.273223929852943)
(23.537472666091563, 25.213059889125326)
(22.014424688119526, 25.439759624925731)
 
Ecart type
(25.160367660781922, 22.978675420675572)
(28.384086244110296, 28.174745066633871)
(27.699922237876009, 28.467917263184447)
 
Amplitude moyenne du pic
(6.8267701909455525, 7.4534204463463691)
(7.0544199737749453, 4.1889993957405895)
(8.5705320160956902, 3.540900698567548)
 
Ecart type
(11.533437502515227, 15.782628498109318)
(11.092538296421248, 7.6679442034126071)
(13.954412452214806, 5.9219791982112397)
 
Vitesse
(852.60887498034469, 917.30464788796996)
(379.81345103853556, 371.27490305180493)
 
Epaisseur
(0.84643250688705229, 0.88865059930403401)
(0.2881564966292019, 0.40298713382719098)


In [171]:
df.head(1)

,Ximf0,Ximf1,Ximf2,Yimf0,Yimf1,Yimf2,coil,speed,sticking,thickness,time_begin,time_end
0,1.283685,6.097503,0.962764,24.797989,6.12363,3.701913,0,697.881497,False,0.6,61.0,62.0


## Avec normalisation

In [166]:
peakDB = 'peaks_0.25t_16_200.h5'
df = concat_df(all_coils, peakDB)
df_compare()

     COLLANT            NON COLLANT  
 
Fréquence moyenne du pic
(18.833744588582061, 16.273223929852943)
(23.537472666091563, 25.213059889125326)
(22.014424688119526, 25.439759624925731)
 
Ecart type
(25.160367660781922, 22.978675420675572)
(28.384086244110296, 28.174745066633871)
(27.699922237876009, 28.467917263184447)
 
Amplitude moyenne du pic
(6.8267701909455525, 7.4534204463463691)
(7.0544199737749453, 4.1889993957405895)
(8.5705320160956902, 3.540900698567548)
 
Ecart type
(11.533437502515227, 15.782628498109318)
(11.092538296421248, 7.6679442034126071)
(13.954412452214806, 5.9219791982112397)
 
Vitesse
(852.60887498034469, 917.30464788796996)
(379.81345103853556, 371.27490305180493)
 
Epaisseur
(0.84643250688705229, 0.88865059930403401)
(0.2881564966292019, 0.40298713382719098)


In [168]:
df

,Ximf0,Ximf1,Ximf2,Yimf0,Yimf1,Yimf2,coil,speed,sticking,thickness,time_begin,time_end
0,25.592870,26.854491,28.116111,23.204153,3.633037,3.752148,12,1022.035352,False,0.77,21.000000,21.250000
1,23.069630,19.284769,38.209074,13.912256,2.116523,2.100310,12,1021.643491,False,0.77,21.250000,21.500000
2,12.976667,18.023148,12.976667,13.457024,3.299008,5.167505,12,1021.927016,False,0.77,21.500000,21.750000
3,24.331250,93.720370,123.999259,4.298913,1.164621,2.459455,12,1021.343652,False,0.77,21.750000,22.000000
4,24.331250,188.341898,84.889028,4.932550,1.578038,1.772237,12,1021.712166,False,0.77,22.000000,22.250000
5,16.761528,19.284769,26.854491,8.014184,2.883518,1.153361,12,1021.647443,False,0.77,22.250000,22.500000
6,45.778796,136.615463,44.517176,2.898339,2.762660,6.527809,12,1022.400410,False,0.77,22.500000,22.750000
7,45.778796,89.935509,110.121435,2.347327,1.369562,3.135894,12,1021.919490,False,0.77,22.750000,23.000000
8,45.778796,161.847870,168.155972,2.661810,0.715097,2.792960,12,1022.032652,True,0.77,23.000000,23.250000
9,45.778796,190.865139,87.412269,2.471116,1.006387,1.579285,12,1021.982119,True,0.77,23.250000,23.500000
